In [1]:
from tifffile import imread
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage
# import cv2
%matplotlib notebook
from mpl_toolkits import mplot3d
import scipy.signal as scps
import pandas as pd
from miniscope_tiffs import scroll_images, get_all_CropROIs, GetMiniscopeDirs
import copy
import pandas
import os
import glob

In [9]:
animals = ['Hipp16942'] # ['Hipp16941', 'Hipp16942', 'Hipp16943']
topdir = 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/'
mini_name = 'msCam.tiff'
behav_name = 'behavCam.tiff'
onlytiff = True # or False or None


thresh = 230 # luminance value tro consider (led >= thresh)
min_size = 3 # minimum size to consider


MiniFolders, BehavFolders, parentMiniFolders, parentBehavFolders = \
    GetMiniscopeDirs(topdir, animals, TiffDirsOnly=True, miniscope_tiff_name=mini_name, behavcam_tiff_name=behav_name)

['C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/Hipp16942/2022_06_10/18_25_10/',
 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/Hipp16942/2022_06_10/18_32_52/',
 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/Hipp16942/2022_06_10/18_44_02/',
 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/Hipp16942/2022_06_13/18_43_34/',
 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/Hipp16942/2022_06_13/19_04_07/',
 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/Hipp16942/2022_06_13/19_18_12/',
 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/Hipp16942/2022_06_14/20_00_14/',
 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/Hipp16942/2022_06_14/20_16_24/',
 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/Hipp16942/2022_06_14/20_21_33/',
 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/Hipp16942/2022_06_14/20

In [ ]:
for sessDir in parentBehavFolders:
    behavDir = str(sessDir + 'BehavCam/')
    vidName = str(behavDir + 'behavCam.tiff')
    outFileName = str(sessDir + 'position_data.csv')
    timestampFile_bh = str(behavDir + 'timeStamps.csv')
    timestampFile_ms = str(sessDir + 'MiniLFOV/timeStamps.csv')
    
    ms_dat = pd.read_csv(timestampFile_ms)
    ms_t = ms_dat['Time Stamp (ms)']
    beh_dat = pd.read_csv(timestampFile_bh)
    beh_t = beh_dat['Time Stamp (ms)']
    
    bmat = imread(vidName)
    frames, d1, d2 = bmat.shape
    numFrames, h, w = bmat.shape
    tds = np.round(len(beh_t)/numFrames).astype(int)
    tds
    beh_t = beh_t[::tds]
    behav_dt = np.median(np.abs(np.diff(beh_t, n=1)))
    behav_dt
    behav_fps = np.round(1000/behav_dt)

In [ ]:
## LOAD THE VIDEO
topdir = 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/Hipp16942/2022_06_18/16_44_33/'
behavDir = str(topdir + 'BehavCam/')
# topdir.replace("\\", "/")
vidName = str(behavDir + 'behavCam.tiff')
outFileName = str(behavDir + 'position_data.csv')
timestampFile = str(behavDir + 'timeStamps.csv')

bmat = imread(vidName)
frames, d1, d2 = bmat.shape
# bLin = bmat.reshape((frames, d1*d2))
bg = np.max(bmat, axis=0).astype('u8')

In [ ]:
t1 = str(topdir + 'MiniLFOV/timestamps.csv')
t2 = str(topdir + 'BehavCam/timestamps.csv')
ms_dat = pd.read_csv(t1)
ms_t = ms_dat['Time Stamp (ms)']
beh_dat = pd.read_csv(t2)
beh_t = beh_dat['Time Stamp (ms)']

numFrames, h, w = bmat.shape
tds = np.round(len(beh_t)/numFrames).astype(int)
tds
beh_t = beh_t[::tds]
behav_dt = np.median(np.abs(np.diff(beh_t, n=1)))
behav_dt
behav_fps = np.round(1000/behav_dt)


In [ ]:
behav_fps

In [ ]:
# Looking at different ways of determining threshold
a999 = np.zeros([frames, 1])
a90 = np.zeros([frames, 1])
a95 = np.zeros([frames, 1])
a99 = np.zeros([frames, 1])

ind = 0
for im in bmat:
    v = im[im>0]
    a999[ind] = np.quantile(v, q=.999)
    a90[ind] = np.quantile(v, q=.9)
    a95[ind] = np.quantile(v, q=.95)
    a99[ind] = np.quantile(v, q=.99)
    
    ind = ind+1
    
plt.plot(a999)
plt.plot(a99)
plt.plot(a95)
plt.plot(a90)

In [ ]:

scroll_images(bmat)


## Parameters

In [ ]:
thresh = 230 # luminance value tro consider (led >= thresh)
min_size = 3 # minimum size to consider

In [ ]:
video_name = '' # expression labelling miniscope avi files, such as 'msCam' (if 'msCam1.avi', 'msCam2.avi',...)
                # or '' (if '0.avi', '1.avi', etc). Starts at the lowest and goes to the highest
crop_shape = 'circle' #'sq' # 'rect' # 'circle', 'rectangle', or 'square'. For no cropping: crop_shape=[]
verbose = False # more or less print details

cropROIs, maskROIs  = get_all_CropROIs([behavDir], avi_names=video_name, shape=crop_shape, verbose=verbose)

In [ ]:
numframes, h, w = bmat.shape
circleDef = maskROIs[behavDir]
center = circleDef[0]
radius = circleDef[1]
Y, X = np.ogrid[:h, :w]
# dist_from_center = np.sqrt((X - center[0]+cropROI[0][0])**2 + (Y-center[1]+cropROI[0][1])**2)
dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
outer_mask = dist_from_center > radius

# doesnt rotate properly
# outer_mask = list(zip(*outer_mask[::-1]))
# outer_mask = list(zip(*outer_mask[::-1]))
outer_mask

In [ ]:
im
# mrthod from https://stackoverflow.com/questions/8421337/rotating-a-two-dimensional-array-in-python
imt1 = list(zip(*im[::-1]))
imt2 = list(zip(*imt1[::-1]))
## SHOULD ROTATE 180 TO ALIGN WITH BSG CAM

imrot = np.hstack((im, imt1, imt2))
plt.figure()
plt.imshow(imrot)

In [12]:
def extract_pos_APA(bmat, thresh=255, min_size=10, outer_mask=None):
    x = np.zeros(frames) # column index
    y = np.zeros(frames) # row index
    arena_x = np.zeros(frames) # column index
    arena_y = np.zeros(frames) # row index
    ind = 0
    frames, h, w = bmat.shape
    if outer_mask is None:
        outer_mask = np.ones((h, w), dtype='uint8')
    for im in bmat:
        # first mask the image by the threshold
        imtemp = copy.deepcopy(im)
        # I want to align it with the BSG caerma, 180 rotation
    #     imtemp = list(zip(*imtemp[::-1])) # 90 degree rotation
    #     imtemp = list(zip(*imtemp[::-1])) # 90 degree rotation

        im_inside = copy.deepcopy(imtemp)
        im_outside = copy.deepcopy(imtemp)
        im_inside[outer_mask==1] = 0
        im_outside[outer_mask==0] = 0

        ## CALCULATE RAT LED POSITION
        mask       = im_inside >= thresh
        label_im, nb_labels = ndimage.label(mask)
        # calculate various image properties of the remaining ROIs
        sizes      = ndimage.sum(mask, label_im, range(nb_labels + 1))
        max_vals   = ndimage.maximum(im, label_im, range(nb_labels + 1))
        mean_vals  = ndimage.mean(im, label_im, range(nb_labels + 1))

        # LED max and mean value must must the greatest of the ROIs
        a = np.max(max_vals)==max_vals
        b = np.max(mean_vals)==mean_vals
        c = sizes>= min_size
        if any(a==b): # testing that it has the max max and the max mean values
            tempind = np.argwhere(a==b)
            good_ind = tempind[0][0]
            # LED size thresholding
            if sizes[good_ind]>=min_size:
                # extract the center of mass
                com = ndimage.center_of_mass(mask, label_im, range(nb_labels + 1))
                y[ind], x[ind] = com[good_ind]  

        ## DO THE SAME FOR THE ARENA POSITION
        mask       = im_outside >= thresh
        label_im, nb_labels = ndimage.label(mask)
        # calculate various image properties of the remaining ROIs
        sizes      = ndimage.sum(mask, label_im, range(nb_labels + 1))
        max_vals   = ndimage.maximum(im, label_im, range(nb_labels + 1))
        mean_vals  = ndimage.mean(im, label_im, range(nb_labels + 1))

        # LED max and mean value must must the greatest of the ROIs
        a = np.max(max_vals)==max_vals
        b = np.max(mean_vals)==mean_vals
        c = sizes>= min_size
        if any(a==b): # testing that it has the max max and the max mean values
            tempind = np.argwhere(a==b)
            good_ind = tempind[0][0]
            # LED size thresholding
            if sizes[good_ind]>=min_size:
                # extract the center of mass
                com = ndimage.center_of_mass(mask, label_im, range(nb_labels + 1))
                arena_y[ind], arena_x[ind] = com[good_ind]  

        ind = ind+1
        return x, y, arena_x, arena_y

In [ ]:
plt.figure()
plt.subplot(2,1,1)
plt.plot(np.arange(0,ind), arena_x[0:ind])
plt.plot(np.arange(0,ind), arena_y[0:ind])


plt.subplot(2,1,2)
plt.plot(np.arange(0,ind), x[0:ind])
plt.plot(np.arange(0,ind), y[0:ind])

In [ ]:
np.mod(smoothing_size,2)

In [ ]:
# half second smoothing size
smoothing_size = np.round(behav_fps/2).astype('int')
if np.mod(smoothing_size,2)==0:
    smoothing_size = smoothing_size+1 # odd size for kernel
    
xi, yi, speed                   = interp_and_smooth(x,y,smoothing_size)
arena_xi, arena_yi, arena_speed = interp_and_smooth(arena_x,arena_y,smoothing_size)

# x1
fig = plt.figure();
# plt.plot(s);
plt.subplot(2,1,1)
plt.plot(x);
plt.plot(y*-1);
plt.plot(xi);
plt.plot(yi*-1);

plt.subplot(2,1,2)
plt.plot(speed);

# plt.plot(t[1::], xd);

In [ ]:
def interp_and_smooth(x,y,smoothing_size):
    xi = x
    yi = y

    a = xi==0
    b = yi==0
    c = a==b
    xq = np.where(a==c)

    a = xi!=0
    b = yi!=0
    c = a==b
    x0 = np.where(a==c)
    xv = xi[x0]
    yv = yi[x0]

    x1 = np.interp(xq, x0[0], xv)
    y1 = np.interp(xq, x0[0], yv)


    xi[xq] = x1[0]
    yi[xq] = y1[0]

    xi = scps.medfilt(xi, kernel_size=smoothing_size)
    yi = scps.medfilt(yi, kernel_size=smoothing_size)

    xt = np.zeros(len(x)+1)
    yt = np.zeros(len(y)+1)
    xt[1::] = xi # increase size by one to account for diff
    yt[1::] = yi # increase size by one to account for diff
    xt[0] = xi[0]
    yt[0] = yi[0]



    xd = np.diff(xt, n=1)
    yd = np.diff(yt, n=1)

    speed = np.sqrt((xd**2 + yd**2))
    return xi, yi, speed

In [ ]:


fig = plt.figure()
ax1 = plt.axes(projection='3d')
# ax.scatter(x, y, beh_t, c='grey', s=2)
# ax.scatter(xi, yi, beh_t, c='red', s=2)
ax1.plot3D(arena_x, arena_y, beh_t, 'gray')

# ax2 = plt.axes(projection='3d')
ax1.plot3D(xi, yi, beh_t, 'r')



In [ ]:
outFileName

In [ ]:
# x = x.flatten(order='C')
# y = y.flatten(order='C')
behav_dt  = np.round(behav_dt).astype( 'uint16')
behav_fps  = np.round(behav_fps).astype( 'uint16')
timestamps  = np.round(beh_t).astype( 'uint16')
x  = np.round(x).astype( 'uint16')
xi = np.round(xi).astype('uint16')
y  = np.round(y).astype( 'uint16')
yi = np.round(yi).astype('uint16')
speed  = speed.astype( 'float32')
arena_x  = np.round(arena_x).astype( 'uint16')
arena_xi = np.round(arena_xi).astype('uint16')
arena_y  = np.round(arena_y).astype( 'uint16')
arena_yi = np.round(arena_yi).astype('uint16')
arena_speed  = arena_speed.astype( 'float32')

D = {'x_raw':x, 'x':xi, 'y_raw':y, 'y':yi, 'speed':speed, 
     'arena_x_raw':arena_x, 'arena_x':arena_xi, 'arena_y_raw':arena_y, 'arena_y':arena_yi, 'arena_speed':arena_speed,
    'behav_dt':behav_dt, 'behav_fps':behav_fps, 'timestamps':timestamps}
DD = pd.DataFrame(D)
DD

In [ ]:

DD.to_csv(outFileName)

In [ ]:
im_all = None
ind = 0
for im in bmat:
    # first mask the image by the threshold
    imtemp = copy.deepcopy(im)
    # I want to align it with the BSG caerma, 180 rotation
#     imtemp = list(zip(*imtemp[::-1])) # 90 degree rotation
#     imtemp = list(zip(*imtemp[::-1])) # 90 degree rotation
    
    im_inside = copy.deepcopy(imtemp)
    im_outside = copy.deepcopy(imtemp)
    im_inside[outer_mask] = 0
    im_outside[outer_mask==False] = 0    
    ## DO THE SAME FOR THE ARENA POSITION
    mask       = im_outside >= thresh
    label_im, nb_labels = ndimage.label(mask)
    # calculate various image properties of the remaining ROIs
    sizes      = ndimage.sum(mask, label_im, range(nb_labels + 1))
    max_vals   = ndimage.maximum(im, label_im, range(nb_labels + 1))
    mean_vals  = ndimage.mean(im, label_im, range(nb_labels + 1))
    
    # LED max and mean value must must the greatest of the ROIs
    a = np.max(max_vals)==max_vals
    b = np.max(mean_vals)==mean_vals
    c = sizes>= min_size
    if any(a==b): # testing that it has the max max and the max mean values
        q = a==b
        v = a==True
        tempind = np.argwhere(q==v)
        good_ind = tempind[0][0]
        # LED size thresholding
        if sizes[good_ind]>=min_size:
            # extract the center of mass
            com = ndimage.center_of_mass(mask, label_im, range(nb_labels + 1))
            arena_y[ind], arena_x[ind] = com[good_ind]            
            
    imh = np.hstack((im, mask*255))
    if im_all is None:
        im_all  = np.empty([bmat.shape[0], im.shape[0], im.shape[1]*2], dtype=np.uint8)
    im_all[ind] = imh
    
    
    
    ind = ind+1

In [ ]:
plt.figure()
plt.imshow(np.hstack((im, outer_mask*255)))

In [ ]:
scroll_images(im_all)